In [0]:
catalog_name = "dmc_tallerfinaldatabricks_mariovento"

schema_silver = "silver"
schema_gold = "gold"

In [0]:
from pyspark.sql.functions import current_timestamp, col

In [0]:
# Carga la tabla 'tiendas' desde el esquema silver
tiendas_silver = spark.table(f"{catalog_name}.{schema_silver}.tiendas")
 
# Crea el DataFrame 'dim_tienda'
dim_tienda = (
    tiendas_silver
    # Crea la columna 'sk_tienda' convirtiendo 'id_tienda' a tipo bigint
    .withColumn("sk_tienda", col("id_tienda").cast("bigint"))
    # Agrega la columna 'creation_date' con la fecha y hora actual
    .withColumn("creation_date", current_timestamp())
    # Selecciona las columnas relevantes para la dimensión tienda
    .select(
        col("sk_tienda"),
        col("id_tienda"),
        col("nombre"),
        col("ciudad"),
        col("region"),
        col("creation_date")
    )
    .orderBy(col("id_tienda"))
)

In [0]:
dim_tienda.write.format("delta").mode("overwrite").saveAsTable(f"{catalog_name}.{schema_gold}.dim_tienda")

In [0]:
%sql
select * from dmc_tallerfinaldatabricks_mariovento.gold.dim_tienda